![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

Short project description. Your bottom line up front (BLUF) insights.

## Business Understanding

The customer of this project is FutureProduct Advisors, a consultancy that helps their customers develop innovative and new consumer products. FutureProduct’s customers are increasingly seeking help from their consultants in go-to-market activities. 

FutureProduct’s consultants can support these go-to-market activities, but the business does not have all the infrastructure needed to support it. Their biggest ask is for a tool to help them find interesting, up-and-coming music to accompany social posts and online ads for go-to-market promotions. 

**Stakeholders**

- FutureProduct Managing Director: oversees their consulting practice and is sponsoring this project.
- FutureProduct Senior Consultants: the actual users of the prospective tool. A small subset of the consultants will pilot the prototype tool.
- My consulting leadership: sponsors of this effort; will provide oversight and technical input of the project as needed.

**Primary Goals**

1.	Build a data tool that can evaluate any song in the Billboard Hot 100 list and make predictions about:
    -	The song’s position on the Hot 100 list 4 weeks in the future
    -	The song’s highest position on the list in the next 6 months
2.	Create a rubric that lists the 3 most important factors for songs’ placement on the Hot 100 list for each hear from 2000 to 2021.


## Data Understanding

Billboard Hot 100 weekly charts (Kaggle): https://www.kaggle.com/datasets/thedevastator/billboard-hot-100-audio-features

I’ve chosen this dataset because it has a direct measurement of song popularity (the Hot 100 list) and because its long history gives significant context to a song’s positioning in a given week.
The features list gives a wide range of song attributes to explore and enables me to determine what features most significantly contribute to a song’s popularity and how that changes over time.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import ast
from collections import Counter

import xgboost as xgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(42)

In [2]:
df_hotlist_all = pd.read_csv('Data/Hot Stuff.csv')
df_features_all = pd.read_csv('Data/Hot 100 Audio Features.csv')

In [3]:
# exploring hotlist df
df_hotlist_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327895 entries, 0 to 327894
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   327895 non-null  int64  
 1   url                     327895 non-null  object 
 2   WeekID                  327895 non-null  object 
 3   Week Position           327895 non-null  int64  
 4   Song                    327895 non-null  object 
 5   Performer               327895 non-null  object 
 6   SongID                  327895 non-null  object 
 7   Instance                327895 non-null  int64  
 8   Previous Week Position  295941 non-null  float64
 9   Peak Position           327895 non-null  int64  
 10  Weeks on Chart          327895 non-null  int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 27.5+ MB


In [4]:
# exploring features df
df_features_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29503 entries, 0 to 29502
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      29503 non-null  int64  
 1   SongID                     29503 non-null  object 
 2   Performer                  29503 non-null  object 
 3   Song                       29503 non-null  object 
 4   spotify_genre              27903 non-null  object 
 5   spotify_track_id           24397 non-null  object 
 6   spotify_track_preview_url  14491 non-null  object 
 7   spotify_track_duration_ms  24397 non-null  float64
 8   spotify_track_explicit     24397 non-null  object 
 9   spotify_track_album        24391 non-null  object 
 10  danceability               24334 non-null  float64
 11  energy                     24334 non-null  float64
 12  key                        24334 non-null  float64
 13  loudness                   24334 non-null  flo

#### Exploratory Data Analysis

## Data Preparation

### Initial Data Selection and Feature Engineering

In [5]:
# removing hotlist df attributes that will not be used in cleaning or analysis
df_hotlist_all = df_hotlist_all.drop(['index', 'url', 'Song', 'Performer'], axis=1)
# converting WeekID to datetime
df_hotlist_all['WeekID'] = pd.to_datetime(df_hotlist_all['WeekID'], errors='coerce')
df_hotlist_all = df_hotlist_all.sort_values(by='WeekID')

# creating a new hotlist df with only complete year data from 2000 - 2020, the time period being studied
df_hotlist_2000s = df_hotlist_all.loc[(df_hotlist_all['WeekID'] > '1999-12-31') & (df_hotlist_all['WeekID'] < '2021-01-01')]

# adding a column to calculate the week over week change in rank
def diff(a, b):
    return a - b

df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s.apply(lambda x: diff(x['Week Position'], x['Previous Week Position']), axis=1)
# replacing NaNs with 0
df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s['Rank_Change'].fillna(0)

# removing features df attributes that will not be used in cleaning or analysis
df_features_all = df_features_all.drop(['index', 'Performer', 'Song', 'spotify_track_album', 
                                        'spotify_track_preview_url', 'spotify_track_explicit', 
                                        'spotify_track_popularity'], axis=1)

C:\Users\marha\AppData\Local\Temp\ipykernel_26296\797317108.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s.apply(lambda x: diff(x['Week Position'], x['Previous Week Position']), axis=1)
C:\Users\marha\AppData\Local\Temp\ipykernel_26296\797317108.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s['Rank_Change'].fillna(0)


In [ ]:
# new df with the max weekly rank change for each song in df_hotlist_2000s
df_max_rank_change = df_hotlist_2000s.groupby('SongID', as_index=False)['Rank_Change'].max()
df_max_rank_change.rename(columns={'Rank_Change': 'Max_Rank_Change'}, inplace=True)
df_max_rank_change.set_index('SongID', inplace=True)

# new df with the max peak rank for each song in df_hotlist_2000s
df_max_peak_pos = df_hotlist_2000s.groupby('SongID', as_index=False)['Peak Position'].max()
df_max_peak_pos.rename(columns={'Peak Position': 'Max_Peak_Position'}, inplace=True)
df_max_peak_pos.set_index('SongID', inplace=True)

# ensuring these new dfs have no null values
df_max_rank_change['Max_Rank_Change'].isna().sum(), df_max_peak_pos['Max_Peak_Position'].isna().sum()

(0, 0)

In [7]:
# extracting full list of songs in the time period being studied
songid_list = df_hotlist_2000s['SongID'].unique()

# creating a features df with only songs in df_hotlist_2000s
df_features_2000s = df_features_all[df_features_all['SongID'].isin(songid_list)]

# checking for duplicates
print(len(df_features_2000s))
print(len(pd.unique(df_features_2000s['SongID'])))

8781
8664


In [8]:
# removing duplicates and rechecking
df_features_2000s = df_features_2000s.drop_duplicates(subset='SongID')

print(len(df_features_2000s))
print(len(pd.unique(df_features_2000s['SongID'])))

8664
8664


In [ ]:
# adding max peak position to features df
df_2000s_data = df_features_2000s.join(df_max_peak_pos, on='SongID')

# adding max rank change to features df
df_2000s_data = df_2000s_data.join(df_max_rank_change, on='SongID')

# removing entries with missing values and defining as a new df
df_cleaned = df_2000s_data[df_2000s_data.notna().all(axis=1)]
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7798 entries, 5 to 29499
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SongID                     7798 non-null   object 
 1   spotify_genre              7798 non-null   object 
 2   spotify_track_id           7798 non-null   object 
 3   spotify_track_duration_ms  7798 non-null   float64
 4   danceability               7798 non-null   float64
 5   energy                     7798 non-null   float64
 6   key                        7798 non-null   float64
 7   loudness                   7798 non-null   float64
 8   mode                       7798 non-null   float64
 9   speechiness                7798 non-null   float64
 10  acousticness               7798 non-null   float64
 11  instrumentalness           7798 non-null   float64
 12  liveness                   7798 non-null   float64
 13  valence                    7798 non-null   float64
 

### Feature Engineering for Genre

The dataset has genre in a single column; the entry for each song has a variety of genres listed in that single column. In order to explore genre, I'll need to break this field out.

In [10]:
# generating a df with unique genre names
unique_genres = list(set(
    genre 
    for genre_string in df_cleaned['spotify_genre'] 
    if pd.notna(genre_string)
    for genre in ast.literal_eval(genre_string)
))

df_unique_genres = pd.DataFrame(unique_genres, columns=['genre'])

# adding counts of each unique genre name
# Extract all genres (with duplicates) and count them
all_genres_list = []
for genre_string in df_cleaned['spotify_genre']:
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        all_genres_list.extend(genre_list)

# Count occurrences
genre_counts = Counter(all_genres_list)

# Map counts to genres dataframe
df_unique_genres['count'] = df_unique_genres['genre'].map(genre_counts)
df_unique_genres = df_unique_genres.sort_values('count', ascending=False)

In [ ]:
# writing to csv for easier review of the data
df_unique_genres.to_csv('genre_counts.csv', index=False)

After reviewing the full set of genre counts, I created a new csv that contains genres which appear in 50 or more song entries.

In [11]:
# loading list of genres with 50 or more instances in df_cleaned
df_genres_50_up = pd.read_csv('genre_counts_50+inst.csv')

# converting df to list
final_genres_list = df_genres_50_up['genre'].tolist()

# manually one-hot encoding each genre

# creating each new genre column and initializing to 0
for genre in final_genres_list:
    df_cleaned[genre] = 0

# iterating through rows to set values to 1 when genre column appears in original spotify_genre column
for idx, genre_string in enumerate(df_cleaned['spotify_genre']):
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        for genre in genre_list:
            df_cleaned.at[idx, genre] = 1

C:\Users\marha\AppData\Local\Temp\ipykernel_26296\3783977166.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[genre] = 0
C:\Users\marha\AppData\Local\Temp\ipykernel_26296\3783977166.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[genre] = 0
C:\Users\marha\AppData\Local\Temp\ipykernel_26296\3783977166.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [12]:
# reviewing full df 
pd.set_option('display.max_columns', None)
df_cleaned.head(3)

,SongID,spotify_genre,spotify_track_id,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Max_Peak_Position,Max_Rank_Change,pop,rap,pop rap,dance pop,post-teen pop,hip hop,trap,contemporary country,country road,country,southern hip hop,modern country rock,atl hip hop,r&b,canadian pop,melodic rap,urban contemporary,pop rock,hollywood,glee club,neo mellow,canadian hip hop,toronto rap,edm,gangster rap,country pop,tropical house,hip pop,modern rock,miami hip hop,latin,electropop,chicago rap,conscious hip hop,viral pop,country dawn,uk pop,dirty south rap,philly rap,detroit hip hop,alternative r&b,post-grunge,oklahoma country,talent show,canadian contemporary r&b,neo soul,boy band,reggaeton,electro house,rock,atl trap,nc hip hop,new orleans rap,emo rap,australian country,alternative metal,canadian metal,canadian rock,nu metal,alternative rock,permanent wave,dance rock,new romantic,new wave,new wave pop,soft rock,synthpop,candy pop,europop,adult standards,brill building pop,easy listening,vocal jazz,dancehall,glam metal,plugg,underground hip hop,deep pop r&b,pop punk,pittsburgh rap,acoustic pop,piano rock,art pop,canadian indie,chamber pop,indie pop,indie rock,slow core,stomp and holler,cali rap,slow game,alternative dance,dance-punk,indietronica,new rave,indie pop rap,comic,texas pop punk,bachata,latin pop,tropical,crunk,metropopolis,baton rouge rap,brooklyn drill,nyc rap,australian pop,punk,east coast hip hop,queens hip hop,west coast trap,g funk,complextro,german techno,new jack swing,escape room,indie r&b,indie soul,dmv rap,memphis hip hop,new jersey rap,british soul,danish pop,scandipop,texas country,idol,rap kreyol,dfw rap,deep southern trap,deep talent show,country rock,redneck,american folk revival,cantautor,latin arena pop,mexican pop,rock en espanol,spanish pop,country gospel,alberta country,canadian contemporary country,canadian country,emo,funk,soul,classic soul,disco,motown,post-disco,quiet storm,lilith,folk-pop,ny roots,ethiopian pop,trap queen,canadian pop punk,canadian punk,pop reggaeton,downtempo,electronic trap,shiver pop,latin hip hop,reggaeton flow,rap metal,socal pop punk,alaska indie,singer-songwriter,sertanejo,sertanejo pop,sertanejo universitario,pixie,pop emo,swedish electropop,swedish pop,garage rock,punk blues,big room,brostep,catstep,electra,funk metal,rap rock,australian dance,christian alternative rock,christian rock,canadian latin,bronx hip hop,hardcore hip hop,lounge,girl group,wrestling,west coast rap,show tunes,etherpop,indie poptimism,belgian dance,belgian pop,eurodance,progressive electro house,modern uplift,australian hip hop,kentucky hip hop,folk,mellow gold,heartland rock,art rock,experimental,experimental rock,melancholia,post-punk,psychedelic rock,jam band,barbadian pop,puerto rican pop,trap latino,deep contemporary country,lds youth,reggae fusion,progressive house,ohio hip hop,arkansas country,blues rock,modern blues rock,small room,bubblegum dance,deep big room,dutch house,smooth jazz,smooth saxophone,christian music,lgbtq+ hip hop,reggaeton colombiano,rap latina,houston rap,modern folk rock,uk americana,alternative hip hop,chicano rap,cartoon,children's music,old school hip hop,bounce,electro,disco house,canadian ccm,christian punk,indiecoustica,ectofolk,irish rock,anthem worship,ccm,christian pop,worship,bassline,social media pop,norwegian hip hop,outlaw country,hawaiian hip hop,vapor trap,bhangra,desi hip hop,desi pop,scottish singer-songwriter,grunge,hard rock,k-pop,k-pop boy group,electropowerpop,neon pop punk,trancecore,album rock,classic rock,glam rock,protopunk,north carolina hip hop,house,uk dance,nu-metalcore,trap soul,italian pop,italo dance,rock-and-roll,rockabilly,groove metal,rap conscient,drill,baroque pop,uk contemporary r&b,celtic rock,harlem hip hop,electronica,nu jazz,trip hop,bow pop,country rap,san diego rap,canadian trap,south african rock,christian indie,moombahton,neo-singer-s

I created two datasets: one containing genre and one without. This will allow me to model this data with and without genre.

In [ ]:
# removing fields used for prep/cleaning but not needed for analysis
df_cleaned = df_cleaned.drop(['SongID', 'spotify_genre', 'spotify_track_id'], axis=1)

# my code added columns for all genres in spotify_genre, removing unwanted columns and creating a clean df with genre
last_col_to_keep_genre = 'emo rap'
df_cleaned_genre = df_cleaned.loc[:, :last_col_to_keep_genre]
# removing NaN rows
df_cleaned_genre = df_cleaned_genre.dropna()

# creating a clean df for analysis without genre
last_col_to_keep_no_genre = 'Max_Rank_Change'
df_cleaned_no_genre = df_cleaned.loc[:, :last_col_to_keep_no_genre]
# removing NaN rows
df_cleaned_no_genre = df_cleaned_no_genre.dropna()

df_cleaned_genre.info(), df_cleaned_no_genre.info()

(    spotify_track_duration_ms  danceability  energy  key  loudness  mode  \
 5                    208186.0         0.613   0.764  2.0    -6.509   1.0   
 13                   331466.0         0.570   0.629  9.0    -7.608   0.0   
 16                   229840.0         0.575   0.434  5.0    -8.193   1.0   
 
     speechiness  acousticness  instrumentalness  liveness  valence    tempo  \
 5        0.1360        0.0527          0.000000     0.197    0.417  160.015   
 13       0.0331        0.5930          0.000136     0.770    0.308  127.907   
 16       0.0312        0.7350          0.000066     0.105    0.348  145.916   
 
     time_signature  Max_Peak_Position  Max_Rank_Change  pop  rap  pop rap  \
 5              4.0                4.0             22.0  0.0  0.0      0.0   
 13             4.0               92.0             14.0  1.0  0.0      0.0   
 16             4.0               39.0              0.0  1.0  0.0      1.0   
 
     dance pop  post-teen pop  hip hop  trap  contempo

### Creating Features and Target Variables

I'm prepping 4 versions for XGBoost and k-NN:

1. Max Peak Position, no genre (1__1 variables)
2. Max Peak Position, with genre (1_2 variables)
3. Max Rank Change, no genre (2_1 variables)
4. Max Rank Change, with genre (2_2 variables)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, no genre
X1_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y1_1 = df_cleaned_no_genre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X1_1_train_scaled = scaler.fit_transform(X1_train)
X1_1_test_scaled = scaler.fit_transform(X1_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, including genre
X1_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y1_2 = df_cleaned_genre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X1_2_train, X1_2_test, y1_2_train, y1_2_test = train_test_split(X1_2, y1_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X1_2_train_scaled = scaler.fit_transform(X1_2_train)
X1_2_test_scaled = scaler.fit_transform(X1_2_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, no genre
X2_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y2_1 = df_cleaned_no_genre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X2_1_train, X2_1_test, y2_1_train, y2_1_test = train_test_split(X2_1, y2_1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X2_1_train_scaled = scaler.fit_transform(X2_1_train)
X2_1_test_scaled = scaler.fit_transform(X2_1_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, including genre
X2_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y2_2 = df_cleaned_genre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X2_2_train, X2_2_test, y2_2_train, y2_2_test = train_test_split(X2_2, y2_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X2_2_train_scaled = scaler.fit_transform(X2_2_train)
X2_2_test_scaled = scaler.fit_transform(X2_2_test)

I'm prepping another 4 versions of the data the deep learning model

1. Max Peak Position, no genre (3__1 variables)
2. Max Peak Position, with genre (3_2 variables)
3. Max Rank Change, no genre (2_1 variables)
4. Max Rank Change, with genre (2_2 variables)

In [ ]:
# Prepare features and target for simple deep learning max_peak_position analysis, no genre
X3_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y3_1 = df_cleaned_no_genre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X3_1_train, X3_1_test, y3_1_train, y3_1_test = train_test_split(X3_1, y3_1, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X3_1_train_final, X3_1_val, y3_1_train_final, y3_1_val = train_test_split(X3_1_train, y3_1_train, test_size=0.2, random_state=42)

# normalizing 
scaler.fit(X3_1_train_final)
X3_1_train_scaled = scaler.transform(X3_1_train_final)
X3_1_val_scaled = scaler.transform(X3_1_val)
X3_1_test_scaled = scaler.transform(X3_1_test)

Training data shape: (4990, 13)
Validation data shape: (1248, 13)
Test data shape: (1560, 13)


In [ ]:
# Prepare features and target for simple deep learning max_peak_position analysis, including genre
X3_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y3_2 = df_cleaned_genre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X3_2_train, X3_2_test, y3_2_train, y3_2_test = train_test_split(X3_2, y3_2, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X3_2_train_final, X3_2_val, y3_2_train_final, y3_2_val = train_test_split(X3_2_train, y3_2_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X3_2_train_final)
X3_2_train_scaled = scaler.transform(X3_2_train_final)
X3_2_val_scaled = scaler.transform(X3_2_val)
X3_2_test_scaled = scaler.transform(X3_2_test)

In [ ]:
# Prepare features and target for simple deep learning max_rank_change analysis, no genre
X4_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y4_1 = df_cleaned_no_genre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X4_1_train, X4_1_test, y4_1_train, y4_1_test = train_test_split(X4_1, y4_1, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X4_1_train_final, X4_1_val, y4_1_train_final, y4_1_val = train_test_split(X4_1_train, y4_1_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X4_1_train_final)
X4_1_train_scaled = scaler.transform(X4_1_train_final)
X4_1_val_scaled = scaler.transform(X4_1_val)
X4_1_test_scaled = scaler.transform(X4_1_test)

Training data shape: (4990, 13)
Validation data shape: (1248, 13)
Test data shape: (1560, 13)


In [ ]:
# Prepare features and target for simple deep learning max_rank_change analysis, including genre
X4_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y4_2 = df_cleaned_genre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X4_2_train, X4_2_test, y4_2_train, y4_2_test = train_test_split(X4_2, y4_2, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X4_2_train_final, X4_2_val, y4_2_train_final, y4_2_val = train_test_split(X4_2_train, y4_2_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X4_2_train_final)
X4_2_train_scaled = scaler.transform(X4_2_train_final)
X4_2_val_scaled = scaler.transform(X4_2_val)
X4_2_test_scaled = scaler.transform(X4_2_test)

## Analysis

Text here

In [ ]:
# XGBoost for max_peak_position, no genre

xgb_model1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model1.fit(X1_train, y1_train)
y1_pred = xgb_model1.predict(X1_test)
y1_pred = np.clip(np.round(y1_pred), 1, 100)

rmse1 = np.sqrt(mean_squared_error(y1_test, y1_pred))
r2_1 = r2_score(y1_test, y1_pred)

print(f'RMSE: {rmse1:.3f}')
print(f'R²: {r2_1:.3f}')

RMSE: 26.079
R²: -0.174


In [34]:
# hyperparameter tuning 1 for max_peak_position

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb1 = GridSearchCV(estimator=xgb_model1,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1.fit(X1_train, y1_train)

print("Best parameters:", grid_search_xgb1.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 6, 'subsample': 0.8}


In [35]:
# hyperparameter tuning 2 for max_peak_position

param_grid2 = {
    'max_depth': [5, 6, 7],
    'learning_rate': [0.005, 0.01, 0.015,],
    'subsample': [0.75, 0.8, 0.85],
    'colsample_bytree': [1.0, 1.1, 1.2],
}

grid_search_xgb1 = GridSearchCV(estimator=xgb_model1,
                            param_grid=param_grid2,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1.fit(X1_train, y1_train)

print("Best parameters:", grid_search_xgb1.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.015, 'max_depth': 5, 'subsample': 0.75}


c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
270 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\sklearn.py", line 1170, in fit
    self._Booster = train(
  File

In [36]:
# hyperparameter tuning 3 for max_peak_position

param_grid3 = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.015],
    'subsample': [0.73, 0.74, 0.75],
    'colsample_bytree': [1.0],
}

grid_search_xgb1 = GridSearchCV(estimator=xgb_model1,
                            param_grid=param_grid3,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1.fit(X1_train, y1_train)

print("Best parameters:", grid_search_xgb1.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.015, 'max_depth': 5, 'subsample': 0.75}


In [37]:
# Extract best/final model for max_peak_position
best_xgb1 = grid_search_xgb1.best_estimator_

# predictions
y1_pred_best = best_xgb1.predict(X1_test)
y1_pred_best = np.clip(np.round(y1_pred_best), 1, 100)

# Evaluate XGBoost model
rmse1_best = np.sqrt(mean_squared_error(y1_test, y1_pred_best))
r2_1_best = r2_score(y1_test, y1_pred_best)

print(f'RMSE: {rmse1_best:.3f}')
print(f'R²: {r2_1_best:.3f}')

RMSE: 23.925
R²: 0.012


In [38]:
# XGBoost for max_rank_change

xgb_model2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model2.fit(X2_train, y2_train)
y2_pred = xgb_model2.predict(X2_test)
y2_pred = np.clip(np.round(y2_pred), 1, 100)

rmse2 = np.sqrt(mean_squared_error(y2_test, y2_pred))
r2_2 = r2_score(y2_test, y2_pred)

print(f'RMSE: {rmse2:.3f}')
print(f'R²: {r2_2:.3f}')

RMSE: 12.816
R²: -0.191


In [39]:
# hyperparameter tuning 1 for max_rank_change

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb2_1 = GridSearchCV(estimator=xgb_model2,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_1.fit(X2_train, y2_train)

print("Best parameters:", grid_search_xgb2_1.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'subsample': 1.0}


In [40]:
# hyperparameter tuning 2 for max_rank_change

param_grid4 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [0.9, 1.0, 1.1],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

grid_search_xgb2_2 = GridSearchCV(estimator=xgb_model2,
                            param_grid=param_grid4,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_2.fit(X2_train, y2_train)

print("Best parameters:", grid_search_xgb2_2.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 2, 'subsample': 0.9}


c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
135 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\xgboost\sklearn.py", line 1170, in fit
    self._Booster = train(
  File

In [41]:
# hyperparameter tuning 3 for max_rank_change

param_grid5 = {
    'max_depth': [1, 2],
    'learning_rate': [0.01],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.9],
        }

grid_search_xgb2_3 = GridSearchCV(estimator=xgb_model2,
                            param_grid=param_grid5,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_3.fit(X2_train, y2_train)

print("Best parameters:", grid_search_xgb2_3.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 2, 'subsample': 0.9}


In [42]:
# Extract best/final model  for max_rank_change
best_xgb2 = grid_search_xgb2_3.best_estimator_

y2_pred_best = best_xgb2.predict(X2_test)
y2_pred_best = np.clip(np.round(y2_pred_best), 1, 100)

# Evaluate XGBoost model
rmse2_best = np.sqrt(mean_squared_error(y2_test, y2_pred_best))
r2_2_best = r2_score(y2_test, y2_pred_best)

print(f'RMSE: {rmse2_best:.3f}')
print(f'R²: {r2_2_best:.3f}')

RMSE: 11.722
R²: 0.004


In [43]:
# k-NN model for max_peak_position

# testing different distance metrics to find optimal approach
metrics = ['euclidean', 'manhattan', 'chebyshev']
k_value = 5

# Dictionary to store results
results = {}

# Loop through list of metrics
for metric in metrics:
        # Create and evaluate model with different metrics and k=5
    knn = KNeighborsClassifier(n_neighbors=k_value, metric=metric)
    # Get cross val scores for model
    cv_scores = cross_val_score(knn, X1_train_scaled, y1_train, cv=5, scoring='accuracy')
    # Store the mean of cv scores as value and metric name as key in results dictionary
    results[metric] = cv_scores.mean()

best_metric = max(results, key=results.get)

print(results)
print(f"\nBest metric: {best_metric} with accuracy: {results[best_metric]:.4f}")

{'euclidean': 0.011286584340476555, 'manhattan': 0.013338451302523157, 'chebyshev': 0.01196961388578155}

Best metric: manhattan with accuracy: 0.0133


In [44]:
# k-NN model for max_rank_change

# testing different distance metrics to find optimal approach
metrics = ['euclidean', 'manhattan', 'chebyshev']
k_value = 5

# Dictionary to store results
results2 = {}

# Loop through list of metrics
for metric in metrics:
        # Create and evaluate model with different metrics and k=5
    knn2 = KNeighborsClassifier(n_neighbors=k_value, metric=metric)
    # Get cross val scores for model
    cv_scores2 = cross_val_score(knn2, X2_train_scaled, y2_train, cv=5, scoring='accuracy')
    # Store the mean of cv scores as value and metric name as key in results dictionary
    results2[metric] = cv_scores2.mean()

best_metric2 = max(results2, key=results2.get)

print(results2)
print(f"\nBest metric: {best_metric2} with accuracy: {results2[best_metric2]:.4f}")

c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\marha\.conda\envs\ai-environment\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


{'euclidean': 0.1793772162634438, 'manhattan': 0.18125814305455024, 'chebyshev': 0.18143039927471066}

Best metric: chebyshev with accuracy: 0.1814


In [45]:
# simple deep learning model for max_peak_position, no regularization or dropout

baseline_model3 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X3_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model3.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline3 = baseline_model3.fit(
    X3_train_scaled, y3_train_final,
    validation_data=(X3_val_scaled, y3_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
156/156 [==============================] - 1s 2ms/step - loss: 2604.7729 - mae: 41.6772 - mse: 2604.7729 - val_loss: 822.8884 - val_mae: 23.2263 - val_mse: 822.8884
Epoch 2/100
156/156 [==============================] - 0s 1ms/step - loss: 742.4853 - mae: 21.8519 - mse: 742.4853 - val_loss: 754.8932 - val_mae: 21.3342 - val_mse: 754.8932
Epoch 3/100
156/156 [==============================] - 0s 1ms/step - loss: 695.0745 - mae: 20.9580 - mse: 695.0745 - val_loss: 720.4818 - val_mae: 21.0505 - val_mse: 720.4818
Epoch 4/100
156/156 [==============================] - 0s 1ms/step - loss: 670.8004 - mae: 20.5292 - mse: 670.8004 - val_loss: 706.4069 - val_mae: 20.6365 - val_mse: 706.4069
Epoch 5/100
156/156 [==============================] - 0s 1ms/step - loss: 651.7724 - mae: 20.1376 - mse: 651.7724 - val_loss: 688.1321 - val_mae: 20.7586 - val_mse: 688.1321
Epoch 6/100
156/156 [==============================] - 0s 1ms/step - loss: 637.9807 - mae: 19.9189 - mse: 637.9807 - val_lo

In [46]:
# simple deep learning model for max_peak_position with batch normalization

bnorm_model3 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X3_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model3.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model3 = bnorm_model3.fit(
    X3_train_scaled, y3_train_final,
    validation_data=(X3_val_scaled, y3_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
156/156 [==============================] - 1s 2ms/step - loss: 5818.2920 - mae: 72.2353 - mse: 5818.2920 - val_loss: 5541.6328 - val_mae: 70.2248 - val_mse: 5541.6328
Epoch 2/100
156/156 [==============================] - 0s 2ms/step - loss: 4664.2012 - mae: 64.0497 - mse: 4664.2012 - val_loss: 3994.9604 - val_mae: 58.9905 - val_mse: 3994.9604
Epoch 3/100
156/156 [==============================] - 0s 2ms/step - loss: 3125.1001 - mae: 51.7439 - mse: 3125.1001 - val_loss: 2249.8411 - val_mae: 43.8084 - val_mse: 2249.8411
Epoch 4/100
156/156 [==============================] - 0s 2ms/step - loss: 1715.8295 - mae: 37.8723 - mse: 1715.8295 - val_loss: 1356.3463 - val_mae: 33.6493 - val_mse: 1356.3463
Epoch 5/100
156/156 [==============================] - 0s 2ms/step - loss: 920.1410 - mae: 27.1137 - mse: 920.1410 - val_loss: 841.3603 - val_mae: 25.3970 - val_mse: 841.3603
Epoch 6/100
156/156 [==============================] - 0s 2ms/step - loss: 651.3304 - mae: 21.5097 - mse: 651

In [47]:
# simple deep learning model for max_peak_position with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3 = reg_model3.fit(
    X3_train_scaled, y3_train_final,
    validation_data=(X3_val_scaled, y3_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
156/156 [==============================] - 1s 2ms/step - loss: 2726.3462 - mae: 43.4871 - mse: 2726.3220 - val_loss: 906.6723 - val_mae: 25.5651 - val_mse: 906.6480
Epoch 2/100
156/156 [==============================] - 0s 2ms/step - loss: 1201.9032 - mae: 28.4806 - mse: 1201.8800 - val_loss: 761.7105 - val_mae: 22.7117 - val_mse: 761.6862
Epoch 3/100
156/156 [==============================] - 0s 2ms/step - loss: 1114.8127 - mae: 27.3995 - mse: 1114.7883 - val_loss: 767.8294 - val_mae: 23.5026 - val_mse: 767.8052
Epoch 4/100
156/156 [==============================] - 0s 2ms/step - loss: 1021.8735 - mae: 26.2133 - mse: 1021.8497 - val_loss: 734.9078 - val_mae: 22.8980 - val_mse: 734.8837
Epoch 5/100
156/156 [==============================] - 0s 2ms/step - loss: 980.4260 - mae: 25.5589 - mse: 980.4018 - val_loss: 758.1610 - val_mae: 23.6975 - val_mse: 758.1369
Epoch 6/100
156/156 [==============================] - 0s 2ms/step - loss: 928.9761 - mae: 24.6999 - mse: 928.9522 - 

In [48]:
# model evaluation

print("=== Baseline Model ===")
train_scores3 = baseline_model3.evaluate(X3_train_scaled, y3_train_final, verbose=0)
val_scores3   = baseline_model3.evaluate(X3_val_scaled, y3_val, verbose=0)
print(f"Train MAE: {train_scores3[1]:.4f}, Train MSE: {train_scores3[2]:.4f}")
print(f"Val   MAE: {val_scores3[1]:.4f}, Val   MSE: {val_scores3[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn3 = bnorm_model3.evaluate(X3_train_scaled, y3_train_final, verbose=0)
val_scores_bn3   = bnorm_model3.evaluate(X3_val_scaled, y3_val, verbose=0)
print(f"Train MAE: {train_scores_bn3[1]:.4f}, Train MSE: {train_scores_bn3[2]:.4f}")
print(f"Val   MAE: {val_scores_bn3[1]:.4f}, Val   MSE: {val_scores_bn3[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg3 = reg_model3.evaluate(X3_train_scaled, y3_train_final, verbose=0)
val_scores_reg3   = reg_model3.evaluate(X3_val_scaled, y3_val, verbose=0)
print(f"Train MAE: {train_scores_reg3[1]:.4f}, Train MSE: {train_scores_reg3[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3[1]:.4f}, Val   MSE: {val_scores_reg3[2]:.4f}")


=== Baseline Model ===
Train MAE: 16.5914, Train MSE: 479.4088
Val   MAE: 20.3131, Val   MSE: 717.1959

=== BatchNorm Model ===
Train MAE: 14.4114, Train MSE: 364.7207
Val   MAE: 20.8759, Val   MSE: 748.8667

=== Regularized Model (L2 + Dropout) ===
Train MAE: 19.9258, Train MSE: 582.5379
Val   MAE: 20.5973, Val   MSE: 641.0394


In [49]:
# simple deep learning model for max_rank_change, no regularization or dropout

baseline_model4 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X4_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model4.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline4 = baseline_model4.fit(
    X4_train_scaled, y4_train_final,
    validation_data=(X4_val_scaled, y4_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
156/156 [==============================] - 1s 2ms/step - loss: 178.0858 - mae: 9.7469 - mse: 178.0858 - val_loss: 166.2321 - val_mae: 9.2950 - val_mse: 166.2321
Epoch 2/100
156/156 [==============================] - 0s 1ms/step - loss: 136.8776 - mae: 8.5183 - mse: 136.8776 - val_loss: 162.7943 - val_mae: 9.2193 - val_mse: 162.7943
Epoch 3/100
156/156 [==============================] - 0s 1ms/step - loss: 134.5475 - mae: 8.4409 - mse: 134.5475 - val_loss: 162.1804 - val_mae: 9.2294 - val_mse: 162.1804
Epoch 4/100
156/156 [==============================] - 0s 1ms/step - loss: 133.5119 - mae: 8.4203 - mse: 133.5119 - val_loss: 161.1011 - val_mae: 9.1268 - val_mse: 161.1011
Epoch 5/100
156/156 [==============================] - 0s 1ms/step - loss: 132.6144 - mae: 8.3874 - mse: 132.6144 - val_loss: 161.0296 - val_mae: 9.1484 - val_mse: 161.0296
Epoch 6/100
156/156 [==============================] - 0s 1ms/step - loss: 132.0050 - mae: 8.3665 - mse: 132.0050 - val_loss: 160.6107 

In [50]:
# simple deep learning model for max_rank_change with batch normalization

bnorm_model4 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X4_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(1)  # Single output for regression
])

bnorm_model4.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model4 = bnorm_model4.fit(
    X4_train_scaled, y4_train_final,
    validation_data=(X4_val_scaled, y4_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
156/156 [==============================] - 1s 2ms/step - loss: 233.6227 - mae: 11.1268 - mse: 233.6227 - val_loss: 243.5388 - val_mae: 10.9345 - val_mse: 243.5388
Epoch 2/100
156/156 [==============================] - 0s 2ms/step - loss: 158.3326 - mae: 8.9277 - mse: 158.3326 - val_loss: 174.2061 - val_mae: 9.0846 - val_mse: 174.2061
Epoch 3/100
156/156 [==============================] - 0s 2ms/step - loss: 133.8251 - mae: 8.2635 - mse: 133.8251 - val_loss: 160.7258 - val_mae: 8.9098 - val_mse: 160.7258
Epoch 4/100
156/156 [==============================] - 0s 2ms/step - loss: 129.6600 - mae: 8.2733 - mse: 129.6600 - val_loss: 158.6989 - val_mae: 9.0649 - val_mse: 158.6989
Epoch 5/100
156/156 [==============================] - 0s 1ms/step - loss: 129.1541 - mae: 8.2716 - mse: 129.1541 - val_loss: 159.2282 - val_mae: 9.2431 - val_mse: 159.2282
Epoch 6/100
156/156 [==============================] - 0s 2ms/step - loss: 127.8631 - mae: 8.2685 - mse: 127.8631 - val_loss: 158.822

In [51]:
# simple deep learning model for max_rank_change with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),


    layers.Dense(1)  # Single output for regression
])

reg_model4.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4 = reg_model4.fit(
    X4_train_scaled, y4_train_final,
    validation_data=(X4_val_scaled, y4_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
156/156 [==============================] - 1s 2ms/step - loss: 176.6633 - mae: 9.6995 - mse: 176.6477 - val_loss: 166.3491 - val_mae: 9.1336 - val_mse: 166.3333
Epoch 2/100
156/156 [==============================] - 0s 2ms/step - loss: 147.0811 - mae: 8.8856 - mse: 147.0652 - val_loss: 164.6426 - val_mae: 9.0389 - val_mse: 164.6268
Epoch 3/100
156/156 [==============================] - 0s 2ms/step - loss: 144.3740 - mae: 8.7961 - mse: 144.3582 - val_loss: 163.3381 - val_mae: 8.9568 - val_mse: 163.3223
Epoch 4/100
156/156 [==============================] - 0s 2ms/step - loss: 142.8192 - mae: 8.7146 - mse: 142.8034 - val_loss: 160.8327 - val_mae: 8.9182 - val_mse: 160.8168
Epoch 5/100
156/156 [==============================] - 0s 1ms/step - loss: 141.2600 - mae: 8.6600 - mse: 141.2443 - val_loss: 161.9067 - val_mae: 8.9346 - val_mse: 161.8910
Epoch 6/100
156/156 [==============================] - 0s 2ms/step - loss: 139.3476 - mae: 8.6014 - mse: 139.3318 - val_loss: 159.2795 

In [52]:
# model evaluation

print("=== Baseline Model ===")
train_scores4 = baseline_model4.evaluate(X4_train_scaled, y4_train_final, verbose=0)
val_scores4   = baseline_model4.evaluate(X4_val_scaled, y4_val, verbose=0)
print(f"Train MAE: {train_scores4[1]:.4f}, Train MSE: {train_scores4[2]:.4f}")
print(f"Val   MAE: {val_scores4[1]:.4f}, Val   MSE: {val_scores4[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn4 = bnorm_model4.evaluate(X4_train_scaled, y4_train_final, verbose=0)
val_scores_bn4   = bnorm_model4.evaluate(X4_val_scaled, y4_val, verbose=0)
print(f"Train MAE: {train_scores_bn4[1]:.4f}, Train MSE: {train_scores_bn4[2]:.4f}")
print(f"Val   MAE: {val_scores_bn4[1]:.4f}, Val   MSE: {val_scores_bn4[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg4 = reg_model4.evaluate(X4_train_scaled, y4_train_final, verbose=0)
val_scores_reg4   = reg_model4.evaluate(X4_val_scaled, y4_val, verbose=0)
print(f"Train MAE: {train_scores_reg4[1]:.4f}, Train MSE: {train_scores_reg4[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4[1]:.4f}, Val   MSE: {val_scores_reg4[2]:.4f}")


=== Baseline Model ===
Train MAE: 7.2072, Train MSE: 94.4592
Val   MAE: 9.6520, Val   MSE: 174.9957

=== BatchNorm Model ===
Train MAE: 7.0902, Train MSE: 91.2168
Val   MAE: 9.7439, Val   MSE: 175.9129

=== Regularized Model (L2 + Dropout) ===
Train MAE: 7.8876, Train MSE: 119.9668
Val   MAE: 8.9351, Val   MSE: 159.3270


## Evaluation

### Business Insight/Recommendation 1

### Business Insight/Recommendation 2

### Business Insight/Recommendation 3

### Tableau Dashboard link

## Conclusion and Next Steps
Text here